<a href="https://colab.research.google.com/github/VatsalBabel/PermutationImportance-for-feature-selection/blob/main/feature_imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import eli5
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.metrics import mean_squared_error
from eli5.sklearn import PermutationImportance
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
original_df = pd.read_csv('/content/gdrive/My Drive/My Projects/Agile Analysis/Algos/agile_wc.csv')

In [4]:
original_df.head()

,Effort,Vi,D,V,Sprint Size,Work days,Team Salary,Act Time
0,156,4.2,0.687,2.7,10,22,230000,63
1,202,3.7,0.701,2.5,10,21,260000,92
2,173,4.0,0.878,3.3,10,22,250000,56
3,331,4.5,0.886,3.8,10,22,300000,86
4,124,4.9,0.903,4.2,10,22,300000,32


In [5]:
y = original_df['Effort']
x = original_df[[x for x in original_df.columns if x.count('Effort')==0]]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
#Prediction without feature_selection
lr = LinearRegression().fit(x_train, y_train)
x_test_predicted = lr.predict(x_test)
print('RMSE: ' + str(np.sqrt(mean_squared_error(y_test, x_test_predicted))))

RMSE: 40.65250748473343


In [8]:
#Applying feature_selection and then predicting
perm = PermutationImportance(lr).fit(x_test, y_test)
sel = SelectFromModel(perm, threshold=0.05, prefit=True)
x_train_new = sel.transform(x_train)
x_test_new = sel.transform(x_test)

lr = LinearRegression().fit(x_train_new, y_train)
print('RMSE: ' + str(np.sqrt(mean_squared_error(y_test, lr.predict(x_test_new)))))

RMSE: 36.52480018123602


In [9]:
feature_names = x_train.columns.tolist()
eli5.show_weights(perm, feature_names=feature_names)

Weight,Feature
1.9104 ± 1.7866,Act Time
0.1459 ± 0.1501,V
0.0988 ± 0.1733,D
0.0047 ± 0.0833,Vi
0.0000 ± 0.0000,Work days
0.0000 ± 0.0000,Sprint Size
-0.0252 ± 0.0477,Team Salary


In [10]:
#Features which were selected from our dataset with 0.05 threshold
x_train.columns[sel.get_support()]

Index(['D', 'V', 'Act Time'], dtype='object')